# Forum Enhacement Dashboard

Sara Piñas García (100472784)

Ángela Durán Pinto (100472766)

Marina Gómez Rey (100472836)

María Ángeles Magro Garrote (100472867)


This dashboard offers a dynamic platform for exploring and analyzing forum discussions with ease and enjoyment. Dive into clusters of forum posts, visualize emotional distributions, and discover valuable insights into trending topics. With interactive features such as sentiment-based post selection and emoji recommendations, users can engage with the forum content.

## Imports

In [1]:
pip install -q dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 56.4 MB/s eta 0:00:00


In [2]:
pip install -q dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 3.5 MB/s eta 0:00:00


In [3]:
pip install -q dash-mantine-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.7 MB/s eta 0:00:00


In [4]:
!pip install jupyter-dash
!pip install dash-bootstrap-components
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


In [5]:
import os
import numpy as np
import pandas as pd
from termcolor import colored
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import pathlib
import networkx as nx
import scipy.sparse as scsp
from collections import Counter

In [6]:
# Common imports
import os
import numpy as np
import pandas as pd
from termcolor import colored
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import pathlib
import networkx as nx
import scipy.sparse as scsp
from collections import Counter


# Figures plotted inside the notebook
%matplotlib inline
# High quality figures
%config InlineBackend.figure_format = 'retina'
# Figures style
plt.style.use('seaborn-whitegrid')
sns.set_style("darkgrid")
sns.color_palette("deep")
# Figues size
plt.rcParams['figure.figsize'] = [8, 6]

#To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

<ipython-input-6-3f442b7a44d5>:20: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [7]:
# To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [8]:
# For fancy table Display
%load_ext google.colab.data_table

In [9]:
import re
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def check_nltk_packages():
  packages = ['punkt','stopwords','omw-1.4','wordnet']

  for package in packages:
    try:
      nltk.data.find('tokenizers/' + package)
    except LookupError:
      nltk.download(package)
check_nltk_packages()

try:
  import lxml
except ModuleNotFoundError:
  %pip install lxml

try:
  import contractions
except ModuleNotFoundError:
  %pip install contractions
  import contractions

from bs4 import BeautifulSoup
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 15.5 MB/s eta 0:00:00


In [10]:
# Common imports
import os
import numpy as np
import pandas as pd
from termcolor import colored
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import pathlib
import networkx as nx
import scipy.sparse as scsp
from collections import Counter


# Figures plotted inside the notebook
%matplotlib inline
# High quality figures
%config InlineBackend.figure_format = 'retina'
# Figures style
plt.style.use('seaborn-whitegrid')
sns.set_style("darkgrid")
sns.color_palette("deep")
# Figues size
plt.rcParams['figure.figsize'] = [8, 6]

#To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

<ipython-input-10-3f442b7a44d5>:20: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [11]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import numpy as np
import plotly.express as px
import dash_mantine_components as dmc
import ast
import sys
import nltk
import pathlib
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from nltk.tokenize import wordpunct_tokenize
from bs4 import BeautifulSoup
import collections
import re
from wordcloud import WordCloud
import base64
from PIL import Image
import io
from joblib import load
from scipy import sparse
from sklearn.cluster import KMeans
import joblib
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from dash.dependencies import Input, Output, State
from wordcloud import WordCloud
import plotly.graph_objects as go
import base64
from PIL import Image
import io
import random
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

<ipython-input-11-bd66d2fb1dc5>:11: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-11-bd66d2fb1dc5>:12: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [12]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
try:
  import contractions
except ModuleNotFoundError:
  %pip install -q contractions
  import contractions

In [14]:
# To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [15]:
# If fa2 is available, it is imported
try:
    from fa2 import ForceAtlas2
    is_fa2 = True

except ModuleNotFoundError:
    if sys.version_info < (3, 9):
        %pip install fa2
        from fa2 import ForceAtlas2
        is_fa2 = True
    else:
        print("WARNING: At the time of writing this notebook, fa2 is not available for python 3.9 or higher")
        print("You can try 'pip install fa2', but it will likely fail")
        print("This is not a major issue. Some cells below that use Fa2 will not be executed, you can run the notebook anyway")
        is_fa2 = False

You can try 'pip install fa2', but it will likely fail
This is not a major issue. Some cells below that use Fa2 will not be executed, you can run the notebook anyway


## Dataset

In [16]:
forum = pd.read_csv('forum_with_coordinates.csv')

corpus_size = len(forum)
forum.head()

,Unnamed: 0,index,scaled_emotional_scores,max_emotion,sentiment,Recommended Emojis,topic_emojis,Post Title,Post Text,clean_post_text,topic_number,SVD_Component_1,SVD_Component_2,clusters,formatted_emojis
0,0,0,"{'Anger': 0.0, 'Anticipation': 0.0, 'Disgust':...",Fear,positive,"😈,😅,🤗","['🤚', '🧑\u200d🤝\u200d🧑', '🙋']",AI is an overblown mess right?,Its not going to change the world or even do ...,go change world even useful right look like in...,9,0.236986,-0.085003,4,🤚🧑‍🤝‍🧑🙋
1,1,1,"{'Anger': 0.0, 'Anticipation': 2.4929573953581...",Disgust,neutral,"🤤,💩,🤮","['💇', '💆', '👌']",bump this thread every time you take a huge shit,i think my toilet knew what was coming. and pr...,think toilet knew come clog,0,0.069041,-0.026614,8,💇💆👌
2,2,2,"{'Anger': 0.0, 'Anticipation': 0.0001495774437...",Anticipation,positive,"😃,😆,😀","['👌', '🙋\u200d♂️', '🙋']",ITT new user names,Post good as heck forum handles in case a lurk...,post good forum handle case lurker want finall...,4,0.122810,0.023633,0,👌🙋‍♂️🙋
3,3,3,"{'Anger': 3.8284839203675343e-05, 'Anticipatio...",Joy,positive,"😙,😅,😁","['🤽\u200d♂️', '🏃\u200d♂️', '👨\u200d❤️\u200d👨']",Unpopular Videogame Opinions: A Midsummer's Ni...,DKC 3: Dixie's Double Trouble was better than ...,dixie double trouble well super world super el...,0,0.068148,-0.021758,8,🤽‍♂️🏃‍♂️👨‍❤️‍👨
4,4,4,"{'Anger': 0.0, 'Anticipation': 0.0, 'Disgust':...",No Emotion,positive,"🥰,🙂,😈","['🏤', '🏣', '🧑\u200d🤝\u200d🧑']",Check out this snake's giant shit,.\nOnly registered members can see post attach...,register member see post attachment,14,0.159714,0.241092,13,🏤🏣🧑‍🤝‍🧑


In [17]:
topic_mapping = {
    0: 'Crime',
    1: 'Wholesome',
    2: 'America Politics',
    3: 'Body',
    4: 'Family and Loss',
    5: 'Movie',
    6: 'Entertainment',
    7: 'Time',
    8: 'Memories',
    9: 'Work and Money',
    10: 'Life',
    11: 'Social Media',
    12: 'Forums',
    13: 'Food',
    14: 'Humanity'
}

In [18]:
forum['topic_number'] = forum['topic_number'].replace(topic_mapping)


In [19]:
#Function to transform the 'topic_emojis' column to the correct format
def transform_emojis(row):
    emojis_str = ""
    if isinstance(row['topic_emojis'], str):
        # Convert the string representation of the list to an actual list
        emojis_list = ast.literal_eval(row['topic_emojis'])
        # Join the emojis into a string
        emojis_str = ''.join(emojis_list)
    return emojis_str

#Apply the function to create a new column
#forum['cleaned_topic_emojis'] = forum['topic_emojis'].apply(transform_emojis, axis=1)
forum['cleaned_topic_emojis'] = forum.apply(transform_emojis, axis=1)

In [21]:
emojis = pd.read_csv('emojis_sentiment_new_scale.csv')
emojis.head()

,emoji,name,clean_post_text,emotional_scores
0,😀,grinning face,grin face,"{""Anger"": 0.0, ""Anticipation"": 0.05, ""Disgust""..."
1,😃,grinning face with big eyes,grin face big eye,"{""Anger"": 0.0, ""Anticipation"": 0.05, ""Disgust""..."
2,😄,grinning face with smiling eyes,grin face smile eye,"{""Anger"": 0.0, ""Anticipation"": 0.05, ""Disgust""..."
3,😁,beaming face with smiling eyes,beam face smile eye,"{""Anger"": 0.0, ""Anticipation"": 0.05, ""Disgust""..."
4,😆,grinning squinting face,grin squint face,"{""Anger"": 0.0, ""Anticipation"": 0.05, ""Disgust""..."


In [22]:
def format_emojis(emojis):
    return "".join(emojis.split(",")) if isinstance(emojis, str) else ""

In [23]:
def format_emojis(emojis):
    return "".join(emojis.split(",")) if isinstance(emojis, str) else ""

In [24]:
import json

# Check if 'emotional_scores' contains strings and convert them to dictionaries
def safe_convert_to_dict(value):
    if isinstance(value, str):
        try:
            return json.loads(value)  # Parse string to dictionary
        except json.JSONDecodeError:
            raise ValueError("Unable to convert to dictionary")
    elif isinstance(value, dict):
        return value
    else:
        raise TypeError("Unexpected data type")

# Apply conversion to 'emotional_scores'
emojis['emotional_scores'] = emojis['emotional_scores'].apply(safe_convert_to_dict)



# Extract emotional scores into individual columns
emotions = ["Anger", "Anticipation", "Disgust", "Fear", "Joy", "Sadness", "Surprise", "Trust"]

# Create new columns for each emotion from the dictionary
for emotion in emotions:
    emojis[emotion] = emojis['emotional_scores'].apply(lambda x: x.get(emotion, 0))



# Drop the original 'emotional_scores' column (optional)
emojis.drop(columns=['emotional_scores'], inplace=True)

# Check the updated DataFrame
print(emojis)

    emoji                             name      clean_post_text  Anger  \
0       😀                    grinning face            grin face    0.0   
1       😃      grinning face with big eyes    grin face big eye    0.0   
2       😄  grinning face with smiling eyes  grin face smile eye    0.0   
3       😁   beaming face with smiling eyes  beam face smile eye    0.0   
4       😆          grinning squinting face     grin squint face    0.0   
..    ...                              ...                  ...    ...   
111     😼               cat with wry smile        cat wry smile    0.0   
112     😽                      kissing cat             kiss cat    0.0   
113     🙀                        weary cat            weary cat    0.0   
114     😿                       crying cat              cry cat    0.0   
115     😾                      pouting cat             pout cat    0.0   

     Anticipation  Disgust  Fear       Joy   Sadness  Surprise     Trust  
0            0.05      0.0   0.0  0.

## PIPELINE

This pipeline encompasses several key stages: **data preprocessing, sentiment analysis, and recommendation generation**.

Initially, forum data is cleaned and preprocessed, including text normalization and removal of irrelevant content such as URLs and numeric digits.

Next, sentiment analysis techniques, with both lexicon-based and machine learning approaches, are applied to obtain the emotional tone of each post.  

Finally, emoji recommendation systems utilize sentiment analysis results to suggest expressive emojis related to the emotional content of each post, enhancing user engagement and communication.

In [25]:
nrc_lexicon_df = pd.read_csv('nrc_lexicon_english_df.csv')

In [26]:
# Assuming filename.csv is the CSV file you want to load
emojis_faces = pd.read_csv('emojis_faces.csv')

In [27]:
# Load TruncatedSVD object
svd_loaded = load('svd_model.pkl')

# Load the sparse matrix
tfidf_sparse_loaded = sparse.load_npz('tfidf_matrix.npz')

In [28]:
# Load the KMeans model
kmeans = joblib.load('best_kmeans_model_15.pkl')

In [29]:
input_file_path = "total_emotions_for_new_text.json"
with open(input_file_path, "r") as json_file:
    total_emotions_for_new_text = json.load(json_file)

In [30]:
# Define stopwords and lemmatizer
stopwords_en = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}
    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)


def clean_text(text):
    """
    Clean and preprocess text data.
    """

    # Make use of BeautifulSoup with the parser "lxml" to get rid of all HTML tags
    text = BeautifulSoup(text, 'lxml').get_text()

    # Use regular expression to remove all URLs in text
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Expand contractions
    text = contractions.fix(text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenization
    tokens = wordpunct_tokenize(text)

    # Homogenization and Lemmatization
    clean_tokens = [wnl.lemmatize(token.lower(), get_wordnet_pos(token)) for token in tokens if token.isalnum()]

    # Remove stopwords
    clean_tokens = [token for token in clean_tokens if token not in stopwords_en]

    # Check if word exists in WordNet corpus
    clean_tokens = [token for token in clean_tokens if wn.synsets(token)]

    # Join tokens back into a string
    clean_text = ' '.join(clean_tokens)

    return clean_text

In [31]:
def calculate_emotional_scores(tokens):
    # Initialize emotional scores
    emotional_scores = {emotion: 0 for emotion in nrc_lexicon_df.columns[1:] if emotion != 'clean_name'}
    tokens = wordpunct_tokenize(tokens)

    # Calculate emotional scores
    for token in tokens:
        # Check if the token appears in the 'clean_name' column of the lexicon DataFrame
        matched_words = nrc_lexicon_df[nrc_lexicon_df['clean_name'] == token]
        if not matched_words.empty:
            # Iterate over the matched words and accumulate emotional scores
            for _ , word_row in matched_words.iterrows():
                for emotion in emotional_scores:
                    # Convert the emotion score to numeric type before adding it
                    emotion_score = pd.to_numeric(word_row[emotion], errors='coerce')
                    if not pd.isnull(emotion_score):
                        emotional_scores[emotion] += emotion_score
    return emotional_scores

In [32]:
def scale_emotional_scores(emotional_scores, total_emotions=None):
    # Calculate total emotions if not provided
    if total_emotions is None:
        total_emotions = {emotion: sum(scores[emotion] for scores in emotional_scores) for emotion in emotional_scores[0].keys()}

    # Create a new list to store scaled emotional scores
    scaled_emotional_scores = []

    # Scale each emotion score
    for scores in emotional_scores:
        scaled_scores = {emotion: scores[emotion] / total_emotions[emotion] for emotion in scores}
        scaled_emotional_scores.append(scaled_scores)

    return total_emotions, scaled_emotional_scores

In [33]:
nltk.download('vader_lexicon')

# Initialize the sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to perform sentiment analysis and return sentiment label
def get_sentiment_label(text):
    # Get the polarity scores from NLTK's SentimentIntensityAnalyzer
    scores = sid.polarity_scores(text)

    # Determine sentiment label based on compound score
    if scores['compound'] > 0.05:
        return 'positive'
    elif scores['compound'] < -0.05:
        return 'negative'
    else:
        return 'neutral'

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [34]:
# Load the KMeans model
kmeans = joblib.load('best_kmeans_model_15.pkl')

In [35]:
# Function to get top N indices with highest values from an array
def top_n_indices(arr, n):
    return np.argpartition(arr, -n)[-n:]

In [36]:
def recommending_emojis(text):

    df_new = pd.DataFrame(columns=['clean_text', 'emotional_scores', 'scaled_emotions_NRC', 'emotions_vader', 'Emojis Sentiment Analysis'])

    # 1. Cleaning the text
    # NECESITAMOS CLEAN_TEXT
    cleaned_text = clean_text(text)

    # 2. Clustering (tfidf, svd, kmeans)
    # NECESITAMOS svd_loaded y tfidf_sparse_loaded
    tfidf_svd_loaded = svd_loaded.transform(tfidf_sparse_loaded)

    predicted_labels = kmeans.predict(tfidf_svd_loaded)
    cluster = predicted_labels[0]

    # 3. Sentiment analysis
    # 3.1. NRC Lexicon
    # NECESITAMOS NRC Y calculate_emotional_scores
    emotional_scores = calculate_emotional_scores(cleaned_text)
    df_new = pd.DataFrame({'clean_text': [cleaned_text], 'emotional_scores': [emotional_scores]})

    # 3.2. Scaling emotional scores
    # NECESITAMOS scale_emotional_scores y total_emotions_for_new_text
    total_emotions, scaled_emotional_scores = scale_emotional_scores(df_new['emotional_scores'], total_emotions=total_emotions_for_new_text)
    df_new["scaled_emotions_NRC"] = scaled_emotional_scores

    # 3.3. Vader
    # NECESITAMOS Vader y get_sentiment_label
    emotions_vader = get_sentiment_label(cleaned_text)
    df_new['emotions_vader'] = emotions_vader

    # 3.4. Emojis for the emotions
    # NECESITAMOS emojis_faces
    df_new_scores = np.array([list(doc['scaled_emotions_NRC'].values()) for _, doc in df_new.iterrows() if doc['scaled_emotions_NRC']])
    if df_new.iloc[0]['emotions_vader'] == 'positive':
        selected_emojis = emojis_faces[emojis_faces['sentiment'] == 'positive']
    elif df_new.iloc[0]['emotions_vader'] == 'negative':
        selected_emojis = emojis_faces[emojis_faces['sentiment'] == 'negative']
    elif df_new.iloc[0]['emotions_vader'] == 'neutral':
        selected_emojis = emojis_faces[emojis_faces['sentiment'] == 'neutral']

    emoji_scores = np.array([list(ast.literal_eval(emoji['scaled_emotional_scores']).values()) for _, emoji in selected_emojis.iterrows()])


    similarities = cosine_similarity(df_new_scores, emoji_scores)
    top_3_indices = np.apply_along_axis(lambda x: top_n_indices(x, 3), axis=1, arr=similarities)
    top_3_emojis = [[selected_emojis.iloc[idx]['emoji'] for idx in indices] for indices in top_3_indices]
    top_3_emojis_str = [','.join(emojis) for emojis in top_3_emojis]
    df_new['Emojis Sentiment Analysis'] = top_3_emojis_str

    return df_new


## DashBoard

In [37]:

# Import necessary libraries
import collections
import io
import base64
import re
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html

# Create Dash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

# Define layout
app.layout = html.Div(style={'backgroundColor': '#f0f2f6', 'padding': '20px'}, children=[
    html.H1([html.Span("😁"), "Forum Enhancement Dashboard", html.Span("📱")], style={'color': '#1e3799', 'fontWeight': 'bold', 'fontSize': '46px', 'textAlign': 'center', 'marginBottom': '30px'}),

    dbc.Row([
        dbc.Col([
            html.Div([
                html.H2("Interactive Clustering Dashboard", style={'color': '#6a89cc', 'marginBottom': '20px'}),
                html.Button("Reset Dashboard", id="reset-button", n_clicks=0, className='btn btn-danger mb-3'),
                html.Div(id='hidden-div', style={'display': 'none'}),
                html.Div([
                    dcc.Graph(id='cluster-plot'),
                    html.Div(id='details')
                ]),
                dcc.Dropdown(
                    id='graph-selector',
                    options=[
                        {'label': 'Emotions Distribution', 'value': 'emotions'},
                        {'label': 'Topics Distribution', 'value': 'topics'},
                        {'label': 'Emoji Cloud', 'value': 'emoji'},
                        {'label': 'Word Cloud', 'value': 'word'}
                    ],
                    value='emotions',
                    clearable=False,
                    className='form-select mb-3'
                ),
                html.Div(id='graph-container')
            ]),
        ], width=12)
    ], style={'margin-bottom': '40px'}),

    dbc.Row([
        dbc.Col(html.H2("Post Selector by Sentiment Topic and Max Emotion 😀😞😐", style={'color': '#6a89cc'}), width=12),
    ], style={'margin-bottom': '20px'}),

    dbc.Row([
        dbc.Col([
            html.Label("Select Sentiment"),
            dcc.Dropdown(
                id='sentiment-dropdown',
                options=[{'label': s, 'value': s} for s in forum['sentiment'].unique()],
                value=None,
                placeholder="Choose a sentiment",
                style={'width': '100%'},
                className='form-select mb-3'
            ),
        ], width=4),

        dbc.Col([
            html.Label("Select Topic"),
            dcc.Dropdown(
                id='topic-dropdown',
                options=[{'label': str(t), 'value': t} for t in forum['topic_number'].unique()],
                value=None,
                placeholder="Choose a topic",
                style={'width': '100%'},
                className='form-select mb-3'
            ),
        ], width=4),

        dbc.Col([
            html.Label("Select Max Emotion"),
            dcc.Dropdown(
                id='max-emotion-dropdown',
                options=[{'label': e, 'value': e} for e in forum['max_emotion'].unique()],
                value=None,
                placeholder="Choose a max emotion",
                style={'width': '100%'},
                className='form-select mb-3'
            ),
        ], width=4),
    ], style={'margin-bottom': '20px'}),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Selected Post"),
                dbc.CardBody(
                    id='post-output',
                    style={'font-size': '16px', 'background-color': 'white'}
                )
            ]),
        ], width=12),
    ], style={'margin-bottom': '20px'}),

        dbc.Row([
        dbc.Col(html.H2("Select an emotion and most frequent emojis for that emotion will appear 😆", style={'color': '#6a89cc'}), width=12),
    ], style={'margin-bottom': '20px'}),
    dcc.Dropdown(
        id="emotion-dropdown",
        options=[
            {"label": emotion, "value": emotion} for emotion in emotions
        ],
        value="Joy",
        clearable=False,
        style={"width": "200px"},
        className='form-select mb-3'
    ),
    dcc.Slider(
        id="num-emojis-slider",
        min=1,
        max=10,
        step=1,
        value=5,
        marks={i: str(i) for i in range(1, 11)},
        tooltip={"placement": "top", "always_visible": True},
        included=True,
        className='mb-3'
    ),
    dcc.Graph(id="emoji-cloud", className='mb-5'),

    html.H2([html.Span("🤔"), 'Recommendation of Emojis by Sentiment Analysis'], style={'textAlign': 'center', 'color': '#1e3799'}),
    dcc.Textarea(
        id='input-text',
        placeholder='Enter your text here...',
        style={'width': '50%', 'height': 100, 'margin': 'auto', 'display': 'block', 'marginTop': '20px', 'marginBottom': '20px'},
        className='form-control'
    ),
    html.Button('Submit', id='submit-button', n_clicks=0, style={'backgroundColor': '#1e3799', 'color': 'white', 'marginTop': '10px', 'marginBottom': '20px'}, className='btn btn-primary'),
    html.Div(id='output-emojis', style={'textAlign': 'center', 'marginTop': '20px'})
])
# Define callback to update cluster plot, details, and graph container based on clickData, graph selection, and reset button
@app.callback(
    [Output('cluster-plot', 'figure'),
     Output('details', 'children'),
     Output('graph-container', 'children'),
     Output('hidden-div', 'children')],
    [Input('cluster-plot', 'clickData'),
     Input('graph-selector', 'value'),
     Input('reset-button', 'n_clicks')],
    [State('hidden-div', 'children')]
)
def update_dashboard(clickData, selected_graph, n_clicks, prev_n_clicks):
    prev_n_clicks = prev_n_clicks or 0

    # Check if reset button is clicked for the first time
    if n_clicks > prev_n_clicks:
        # Reset cluster plot
        fig = px.scatter(forum, x='SVD_Component_1', y='SVD_Component_2', color='clusters', hover_data=['Post Title'])
        fig.update_layout(title="Cluster Visualization", clickmode='event+select')

        # Reset details display
        details_html = html.Div()

        # Reset graph container
        graph_container = html.Div()

        return fig, details_html, graph_container, n_clicks

    # Initialize default outputs
    fig = px.scatter(forum, x='SVD_Component_1', y='SVD_Component_2', color='clusters', hover_data=['Post Title'])
    fig.update_layout(title="Cluster Visualization", clickmode='event+select')
    details_html = html.Div()
    graph_container = html.Div()

    # Check if clickData is not None
    if clickData:
        clicked_point = clickData['points'][0]
        clicked_x = clicked_point['x']
        clicked_y = clicked_point['y']

        # Find the row(s) with matching coordinates
        clicked_rows = forum[
            (forum['SVD_Component_1'] == clicked_x) &
            (forum['SVD_Component_2'] == clicked_y)
        ]

        if not clicked_rows.empty:
            # For simplicity, consider the first matching row
            selected_entry = clicked_rows.iloc[0]

            # Update scatter plot
            filtered_df = forum[forum['clusters'] == selected_entry['clusters']]
            fig = px.scatter(filtered_df, x='SVD_Component_1', y='SVD_Component_2', color='clusters', hover_data=['Post Title'])
            fig.update_layout(title="Cluster Visualization", clickmode='event+select')

            # Create details display
            details_html = html.Div([
                html.H2('Selected Entry Details'),
                html.P(f"Post Title: {selected_entry['Post Title']}"),
                html.P(f"Post Text: {selected_entry['Post Text']}"),
                html.P(f"Cluster: {selected_entry['clusters']}"),
                html.P(f"Recommended Emojis: {selected_entry['Recommended Emojis']}"),
                html.P(f"Formatted Emojis: {selected_entry['formatted_emojis']}"),
            ])

            # Update graph container based on the selected graph
            if selected_graph == 'emotions':
                fig_graph = px.histogram(filtered_df, x='max_emotion', title='Emotions Distribution')
                graph_container = dcc.Graph(figure=fig_graph)

            elif selected_graph == 'topics':
                fig_graph = px.histogram(filtered_df, x='topic_number', title='Topics Distribution')
                graph_container = dcc.Graph(figure=fig_graph)

            elif selected_graph == 'emoji':
                # Extract emojis and calculate frequencies for the selected cluster
                cluster_emojis = []
                for emoji_str in filtered_df['Recommended Emojis']:
                    cluster_emojis.extend(re.findall(r'[^\w\s,]', emoji_str))  # Extract emojis using regex
                cluster_emoji_counter = collections.Counter(cluster_emojis)
                common_cluster_emojis = cluster_emoji_counter.most_common()

                # Display common emojis for the selected cluster as individual elements
                max_cluster_freq = common_cluster_emojis[0][1] if common_cluster_emojis else 1
                emoji_tags = []
                for emoji, freq in common_cluster_emojis:
                    size = 20 + 30 * (freq / max_cluster_freq)
                    emoji_tags.append(html.Span(emoji, style={'font-size': f'{size}px', 'padding': '5px'}))

                graph_container = html.Div(emoji_tags)

            elif selected_graph == 'word':
                # Generate word cloud for the selected cluster
                cluster_text = ' '.join(filtered_df['clean_post_text'])  # Combine text of all entries in the cluster
                wordcloud = WordCloud(width=800, height=400).generate(cluster_text)

                # Convert word cloud image to base64 format for display in Dash
                img_data = wordcloud.to_image()
                img_bytes = io.BytesIO()
                img_data.save(img_bytes, format='PNG')
                img_base64 = base64.b64encode(img_bytes.getvalue()).decode()

                # Display word cloud
                word_cloud_html = html.Div([
                    html.H2('Word Cloud'),
                    html.Img(src='data:image/png;base64,{}'.format(img_base64))
                ])
                graph_container = word_cloud_html

    # If no cluster is clicked, display graphs based on the entire dataset
    else:
        if selected_graph == 'emotions':
            fig_graph = px.histogram(forum, x='max_emotion', title='Emotions Distribution')
            graph_container = dcc.Graph(figure=fig_graph)

        elif selected_graph == 'topics':
            fig_graph = px.histogram(forum, x='topic_number', title='Topics Distribution')
            graph_container = dcc.Graph(figure=fig_graph)

        elif selected_graph == 'emoji':
            # Extract emojis and calculate frequencies for the entire dataset
            emojis = []
            for emoji_str in forum['Recommended Emojis']:
                emojis.extend(re.findall(r'[^\w\s,]', emoji_str))  # Extract emojis using regex
            emoji_counter = collections.Counter(emojis)
            common_emojis = emoji_counter.most_common()

            # Display common emojis for the entire dataset as individual elements
            max_freq = common_emojis[0][1] if common_emojis else 1
            emoji_tags = []
            for emoji, freq in common_emojis:
                size = 20 + 30 * (freq / max_freq)
                emoji_tags.append(html.Span(emoji, style={'font-size': f'{size}px', 'padding': '5px'}))

            graph_container = html.Div(emoji_tags)

        elif selected_graph == 'word':
            # Generate word cloud for the entire dataset
            all_text = ' '.join(forum['clean_post_text'])  # Combine text of all entries
            wordcloud = WordCloud(width=800, height=400).generate(all_text)

            # Convert word cloud image to base64 format for display in Dash
            img_data = wordcloud.to_image()
            img_bytes = io.BytesIO()
            img_data.save(img_bytes, format='PNG')
            img_base64 = base64.b64encode(img_bytes.getvalue()).decode()

            # Display word cloud
            word_cloud_html = html.Div([
                html.H2('Word Cloud'),
                html.Img(src='data:image/png;base64,{}'.format(img_base64))
            ])
            graph_container = word_cloud_html

    return fig, details_html, graph_container, n_clicks


# Define callback to update the post output
@app.callback(
    Output('post-output', 'children'),  # The component to update
    [Input('sentiment-dropdown', 'value'),
     Input('topic-dropdown', 'value'),
     Input('max-emotion-dropdown', 'value')]
)
def display_random_post(selected_sentiment, selected_topic, selected_emotion):
    if selected_sentiment is None or selected_topic is None or selected_emotion is None:
        return html.P("Please select sentiment, topic, and max emotion.")

    # Filter the dataframe based on sentiment, topic, and max emotion
    filtered_df = forum[
        (forum['sentiment'] == selected_sentiment) &
        (forum['topic_number'] == selected_topic) &
        (forum['max_emotion'] == selected_emotion)
    ]

    if filtered_df.empty:
        return html.P("No post matches the selected criteria.")

    # Get a random post from the filtered dataframe
    example_post = filtered_df.sample(n=1).iloc[0]

    return html.Div([
        html.H3("Post Example"),
        html.P(f"Text: {example_post['Post Text']}"),
        html.P(f"Recommended Emojis: {format_emojis(example_post['Recommended Emojis'])}",
               style={'font-size': '30px'}),  # Large font for emojis
        html.P(f"Topic Emojis: {format_emojis(example_post['cleaned_topic_emojis'])}",
               style={'font-size': '30px'}),  # Larger font for better visibility

    ], style={'padding': '10px'})  # Extra padding for cleaner presentation

emoji_scale_factor = 1200
# Define callback to update the emoji cloud
@app.callback(
    Output("emoji-cloud", "figure"),
    [Input("emotion-dropdown", "value"), Input("num-emojis-slider", "value")],
)
def update_emoji_cloud(selected_emotion, num_emojis):
    # Filter and sort by selected emotion
    filtered_df = emojis[["emoji", selected_emotion]].copy()
    filtered_df = filtered_df[filtered_df[selected_emotion] > 0]  # Keep only scores > 0
    filtered_df = filtered_df.sort_values(by=selected_emotion, ascending=False)  # Sort by score
    filtered_df = filtered_df.head(num_emojis)  # Limit to the top N emojis

    # Create a scatter plot with emojis, where size represents the emotion score
    fig = go.Figure(
        go.Scatter(
            x=filtered_df["emoji"],  # X-axis is the emoji
            y=[1] * len(filtered_df),  # Y-axis is constant (for positioning)
            mode="text",  # Display text (emojis)
            text=filtered_df["emoji"],  # The text to display is the emoji
            textfont=dict(size=filtered_df[selected_emotion] * emoji_scale_factor),  # Scale by emotion score
        )
    )

    fig.update_layout(
        yaxis=dict(showticklabels=False, zeroline=False),
        xaxis=dict(showticklabels=True, zeroline=False),
        showlegend=False,
        title=f"Emoji Cloud for {selected_emotion}",
    )

    return fig

@app.callback(
    Output('output-emojis', 'children'),
    [Input('submit-button', 'n_clicks')],
    [State('input-text', 'value')]
)
def update_output(n_clicks, input_text):
    if n_clicks > 0 and input_text:
        # Call recommending_emojis function with the input text
        df_new = recommending_emojis(input_text)
        # Select the Emojis Sentiment Analysis column
        emojis_analysis = df_new['Emojis Sentiment Analysis'].iloc[0]
        # Display the recommended emojis
        emojis_html = [html.Span(emoji, style={'fontSize': '24px', 'padding': '5px'}) for emoji in emojis_analysis.split(',')]
        return emojis_html
    else:
        return ''  # Initial state or no input text

# Run the app with external display mode -> PESTAÑA NUEVA
app.run_server(mode='external')


/usr/local/lib/python3.10/dist-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:


<IPython.core.display.Javascript object>